# Lesson 5: Extracting Genes

© 2019 David Gold. Except where the source is noted, this work is licensed under a [Creative Commons Attribution License CC-BY 4.0](https://creativecommons.org/licenses/by/4.0/).

> Are you taking the class with Dr. Gold?
If so, check for any changes in the course material on GitHub:


## Where we left off with Lesson 4

In [ ]:
cd ~/git/Gold_Lab_Training/Additional_Materials
head Results.txt

Terminal should return the following:

    qseqid	sseqid	pident	length	mismatch	gapopen	qstart	qend	sstart	send	evalue	bitscore
    Ectocarpus_siliculosus_CBN76684.1_Sterol_methyltransferase	sp|P25087|ERG6_YEAST_Sterol_24-C-methyltransferase	43.296	358	182	3	75	413	13	368	2.78e-100	301

Here are the results in a table view for easier interpretation:

|qseqid|sseqid|pident|length|mismatch|gapopen|qstart|qend|sstart|send|evalue|bitscore|
|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|:---|
|Ectocarpus_siliculosus_CBN76684.1_Sterol_methyltransferase|sp\|P25087\|ERG6_YEAST_Sterol_24-C-methyltransferase|43.296|358|182|3|75|413|13|368|2.78e-100|301

This is what the header IDs mean:

|Column header|Meaning|
|:--:|:--:|
|qseqid |query (e.g., gene) sequence id|
|sseqid|subject (e.g., reference genome) sequence id|
|pident|percentage of identical matches|
|length|alignment length|
|mismatch|number of mismatches|
|gapopen|number of gap openings|
|qstart|start of alignment in query|
|qend|end of alignment in query|
|sstart|start of alignment in subject|
|send|end of alignment in subject|
|evalue|expect value|
|bitscore|bit score|

As a reminder, one of our query seqeunces (Ectocarpus_siliculosus_CBN76684.1_Sterol_methyltransferase) had a single hit in the yeast database (sp|P25087|ERG6_YEAST_Sterol_24-C-methyltransferase). If we want to see what the sequence of this BLAST hit looks like, we could open the relevent fasta file up in BBEdit and search for it, but that won't work with very large fasta files. Instead we will use a program called [__Samtools__](http://www.htslib.org/). Samtools is primarily use for working with SAM and BAM files, a common text format for high-throughput sequencing data. But it has some useful tools for working with fasta files more broadly.

## Installing Samtools

We can install Samtools with Homebrew:

In [ ]:
brew install samtools

## Retrieving genetic data with Samtools

Before we can extract data from a fasta file using Samtools, we need to __index__ the file first. We can do that with Samtools' `faidx` command:

In [ ]:
samtools faidx Lesson_4_Yeast_Proteome.fasta

There will now be an index file (Lesson_4_Yeast_Proteome.fasta.fai) added to your folder. This allows Samtools to easily navigate fasta files of any size.

To look at the gene we found with BLAST, we also use the `faidx` command. But this time we add the name of the seqeunce we're interested in
(make sure to wrap the name in quotes):

In [ ]:
samtools faidx Lesson_4_Yeast_Proteome.fasta "sp|P25087|ERG6_YEAST_Sterol_24-C-methyltransferase"

Terminal will return the relevent amino acid sequence:

    >sp|P25087|ERG6_YEAST_Sterol_24-C-methyltransferase
    MSETELRKRQAQFTRELHGDDIGKKTGLSALMSKNNSAQKEAVQKYLRNWDGRTDKDAEE
    RRLEDYNEATHSYYNVVTDFYEYGWGSSFHFSRFYKGESFAASIARHEHYLAYKAGIQRG
    DLVLDVGCGVGGPAREIARFTGCNVIGLNNNDYQIAKAKYYAKKYNLSDQMDFVKGDFMK
    MDFEENTFDKVYAIEATCHAPKLEGVYSEIYKVLKPGGTFAVYEWVMTDKYDENNPEHRK
    IAYEIELGDGIPKMFHVDVARKALKNCGFEVLVSEDLADNDDEIPWYYPLTGEWKYVQNL
    ANLATFFRTSYLGRQFTTAMVTVMEKLGLAPEGSKEVTAALENAAVGLVAGGKSKLFTPM
    MLFVARKPENAETPSQTSQEATQ
    
If you want to save this sequence you can redirect it to an output file (we'll call it ERG6.fasta) with the `>` command:

In [ ]:
samtools faidx Lesson_4_Yeast_Proteome.fasta "sp|P25087|ERG6_YEAST_Sterol_24-C-methyltransferase" > ERG6.fasta

Perhaps you only want to see the part of the yeast gene that matched with the query. Using the `sstart` (start of alignment in subject) and `send` (end of alignment in subject) columns, you'll see that the match encompases amino acids \#13-368. 

We can extract these particular sequences using the previous `samtools faidx` command, but adding a colon (`:`) followed by the region of interest: 

In [ ]:
samtools faidx Lesson_4_Yeast_Proteome.fasta "sp|P25087|ERG6_YEAST_Sterol_24-C-methyltransferase":13-368

Terminal will return the follwing:

    >sp|P25087|ERG6_YEAST_Sterol_24-C-methyltransferase:13-368
    FTRELHGDDIGKKTGLSALMSKNNSAQKEAVQKYLRNWDGRTDKDAEERRLEDYNEATHS
    YYNVVTDFYEYGWGSSFHFSRFYKGESFAASIARHEHYLAYKAGIQRGDLVLDVGCGVGG
    PAREIARFTGCNVIGLNNNDYQIAKAKYYAKKYNLSDQMDFVKGDFMKMDFEENTFDKVY
    AIEATCHAPKLEGVYSEIYKVLKPGGTFAVYEWVMTDKYDENNPEHRKIAYEIELGDGIP
    KMFHVDVARKALKNCGFEVLVSEDLADNDDEIPWYYPLTGEWKYVQNLANLATFFRTSYL
    GRQFTTAMVTVMEKLGLAPEGSKEVTAALENAAVGLVAGGKSKLFTPMMLFVARKP

Compare it to the previous result. Notice how the first 12 amino acids have been trimmed from the sequence.

## Extracting conserved regions from multiple genes with Samtools

In the "Additional_Materials" folder you should find a file called "Lesson_5_Query_Domain.fasta". It contains the following oxidoreductase (an enzyme that catalyzes the transfer of electrons from one molecule to another) from the bacteria *Escherichia coli*:

    >Ecoli_oxidoreductase
    MKKIPLGTTDITLSRMGLGTWAIGGGPAWNGDLDRQICIDTILEAHRCGINLIDTAPGYN
    FGNSEVIVGQALKKLPREQVVVETKCGIVWERKGSLFNKVGDRQLYKNLSPESIREEVEA
    SLQRLGFDYIDIYMTHWQSVPPFYTPIAETVAVLNELKAEGKIRAIGAANVDADHIREYL
    QYGELDIIQAKYSILDRAMENELLPLCRDNGIVVQVYSPLEQGLLTGTITRDYVPGGARA
    NKVWFQRENMLKVIDMLEQWQPLCARYQCTIPTLALAWILKQSDLISILSGATAPEQVRE
    NVAALNINLSDADATLMREMAEVLGR

Let's use BLAST to look for homologous genes in our yeast database:

In [ ]:
cd ~/git/Gold_Lab_Training/Additional_Materials
blastp -query Lesson_5_Query_Domain.fasta -db Yeast -outfmt 6 -evalue 10e-5 -out Lesson_5_BLAST_Results.txt

And once BLAST is finished, add the header information:

In [ ]:
echo -e "qseqid\tsseqid\tpident\tlength\tmismatch\tgapopen\tqstart\tqend\tsstart\tsend\tevalue\tbitscore"|cat \
- Lesson_5_BLAST_Results.txt > /tmp/out && mv /tmp/out Lesson_5_BLAST_Results.txt